# Part 2
In part 1 I created a logistic regressor to predict if a loan will default. In part two I will use a much larger data set to not just predict if a loan will default, but how much money the loan will make.

# Section 1 Getting the Data
I will use the google sheets api to read the data from my google sheets account. 

In [19]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from apiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools
import pandas as pd

In [20]:
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']
creds = None
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('drive', 'v3', credentials=creds)

In [21]:
results = service.files().list(
        pageSize=10, fields="nextPageToken, files(id, name)").execute()
items = results.get('files', [])
print(items)

[{'id': '1OaOA9tJwGzocDX2DFEySKOpuYY9YKCCW', 'name': 'mulas_32161.gpkg'}, {'id': '1QZdZd1-mwMuxgC9lg81ysCt4feOhabVH', 'name': 'QGIS'}, {'id': '1c-W0LpXcUFsYfxJPJ5o9z9hIdZA4s7yQ', 'name': 'mulas.qgz'}, {'id': '1WhBDt3lFHVB9ofv7Io831xJG7TcgRMSUa18AtEEfCR0', 'name': 'EWB Rwanda Homework Assignments'}, {'id': '1zlstkIgmODpFIvg9yw8tJLFeBGhhqJb9plceoH-APxI', 'name': 'EWB Rwandan University Flyer'}, {'id': '1Q4UWpCMIYV59bVUd4ESHhJZiZ13FBSjweTBm_alwJfI', 'name': 'EWB meeting times'}, {'id': '1l1ZOY151dt0zBoLtVxWWWm0EMz-2lnVFNPLDORJiBrg', 'name': 'Mulas Rough Draft Work Plan'}, {'id': '1Pi5Gt4L7K6nTQPeMHGGWc4e5q8OmSOkL2fuYbr_NGHU', 'name': 'Meeting #10 '}, {'id': '1ssZVruY4h2msXieeMPqbQDpOxyxAuYk_rlWAlCcnrCU', 'name': 'Week By Week Master Plan Per Sub-Team Fall 2019- Spring 2020'}, {'id': '1x4yIoeQ3q139FpxKtsoYdHzCuF4a0LwEIDoZ05G12ns', 'name': 'Travel Team Interview Questions'}]


In [22]:
project_folderID='1GW7_97iW8Rs8lPwxcDusPMmirRsyM4P1'

In [23]:
csv_q="mimeType='csv' and parents in '{}'".format(project_folderID)

In [24]:
pageToken=None
response = service.files().list(q=csv_q,spaces='drive',fields='nextPageToken, files(id, name)').execute()
print(response)

{'files': []}


In [25]:
 for file in response.get('files', []):
        print( 'Found file: %s (%s)' % (file.get('name'), file.get('id')))
        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break

In [26]:
def gsheet2df(gsheet):
    """ Converts Google sheet data to a Pandas DataFrame.
    Note: This script assumes that your data contains a header file on the first row!
    Also note that the Google API returns 'none' from empty cells - in order for the code
    below to work, you'll need to make sure your sheet doesn't contain empty cells,
    or update the code to account for such instances.
    """
    header = gsheet.get('values', [])[0]   # Assumes first line is header!
    values = gsheet.get('values', [])[1:]  # Everything else is data.
    if not values:
        print('No data found.')
    else:
        all_data = []
        for col_id, col_name in enumerate(header):
            column_data = []
            for row in values:
                column_data.append(row[col_id])
            ds = pd.Series(data=column_data, name=col_name)
            all_data.append(ds)
        df = pd.concat(all_data, axis=1)
        return df

# Section 2 Data Cleaning

In [27]:
df = pd.read_csv("LoanStats_securev1_2017Q4.csv",skiprows=[0])
print(df.columns)
df.head()

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'orig_projected_additional_accrued_interest',
       'hardship_payoff_balance_amount', 'hardship_last_payment_amount',
       'debt_settlement_flag', 'debt_settlement_flag_date',
       'settlement_status', 'settlement_date', 'settlement_amount',
       'settlement_percentage', 'settlement_term'],
      dtype='object', length=150)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,126364716,NaN,14875.0,14875.0,14875.0,36 months,7.35%,461.69,A,A4,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
1,126379308,NaN,16000.0,16000.0,16000.0,60 months,12.62%,360.95,C,C1,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
2,126390441,NaN,6000.0,6000.0,6000.0,36 months,7.35%,186.23,A,A4,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
3,126410270,NaN,25000.0,25000.0,25000.0,60 months,30.17%,811.45,F,F3,...,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN
4,126413271,NaN,25000.0,25000.0,25000.0,60 months,12.62%,563.98,C,C1,...,NaN,NaN,NaN,Y,Oct-2019,ACTIVE,Oct-2019,9852.0,50.0,18.0


In [29]:
df['credit_score']=(df['fico_range_high']+df['fico_range_low'])/2.
df_simple = df
df_simple['loan_status']=df_simple['loan_status'].replace('Default','Charged Off')
df_simple['loan_status']=df_simple['loan_status'].replace('In Grace Period','Charged Off')

In [32]:
predictorTitles=['credit_score','annual_inc','total_bal_ex_mort','installment','emp_length','revol_util','sub_grade','dti','installment','loan_amnt']
df_simple[predictorTitles].head()

,credit_score,annual_inc,total_bal_ex_mort,installment,emp_length,revol_util,sub_grade,dti,installment,loan_amnt
0,722.0,52956.0,31920.0,461.69,2 years,32%,A4,23.48,461.69,14875.0
1,712.0,55000.0,16493.0,360.95,7 years,29.4%,C1,13.02,360.95,16000.0
2,762.0,65000.0,6100.0,186.23,1 year,17.7%,A4,8.35,186.23,6000.0
3,682.0,354000.0,210231.0,811.45,10+ years,73.5%,F3,20.77,811.45,25000.0
4,727.0,105000.0,116607.0,563.98,1 year,49.4%,C1,24.75,563.98,25000.0


In [ ]:
def emp_length_map(x):
    if type(x) == str:
        x=x.replace(' years','')
        x=x.replace(' year','')
        x=x.replace('+','')
        x=x.replace('>','')
        x=x.replace('<','')
        x=x.replace(' ','')
    return x

In [ ]:
def grade_recoding(x):
    if type(x) == str:
        x=x.replace('A1','34')
        x=x.replace('A2','33')        
        x=x.replace('A3','32')        
        x=x.replace('A4','31') 
        x=x.replace('A5','30')        
        x=x.replace('B1','29')        
        x=x.replace('B2','28')        
        x=x.replace('B3','27')        
        x=x.replace('B5','26') 
        x=x.replace('B4','25')        
        x=x.replace('C1','24')        
        x=x.replace('C2','23')        
        x=x.replace('C3','22')        
        x=x.replace('C4','21')        
        x=x.replace('C5','20') 
        x=x.replace('D1','19')        
        x=x.replace('D2','18')        
        x=x.replace('D3','17')        
        x=x.replace('D4','16')        
        x=x.replace('D5','15') 
        x=x.replace('E1','14')        
        x=x.replace('E2','13')        
        x=x.replace('E3','12')        
        x=x.replace('E4','11')        
        x=x.replace('E5','10')
        x=x.replace('F1','9')        
        x=x.replace('F2','8')        
        x=x.replace('F3','7')        
        x=x.replace('F4','6')        
        x=x.replace('F5','5')
        x=x.replace('G1','4')        
        x=x.replace('G2','3')        
        x=x.replace('G3','2')        
        x=x.replace('G4','1')        
        x=x.replace('G5','1')
    return x

In [ ]:
def revol_util_map(x):
    if type(x) == str:
        x=x.replace('%','')
        x=float(x)
    return x

In [ ]:
def dti_adjustment(dti,income,amount):
    return (dti/100*income+amount)/income